[KMeans clustering in PySpark](https://stackoverflow.com/questions/47585723/kmeans-clustering-in-pyspark]) -- StackOverflow

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark.sparkContext.setLogLevel("OFF")

24/11/09 07:23:20 WARN Utils: Your hostname, Ambujs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.86.3.215 instead (on interface en0)
24/11/09 07:23:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/09 07:23:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Create the DataFrame

In [2]:
df = spark.createDataFrame([
    ("Delhi", 28.7, 77.1),   
    ("Gurgaon", 28.5, 77.0),
    ("Faridabad", 28.4, 77.3),
    ("Noida", 28.5, 77.4),

    ("Manipal", 13.4, 74.8),
    ("Udupi", 13.3, 74.7)
], schema="city string, latitude double, longitude double")

df.show()

+---------+--------+---------+
|     city|latitude|longitude|
+---------+--------+---------+
|    Delhi|    28.7|     77.1|
|  Gurgaon|    28.5|     77.0|
|Faridabad|    28.4|     77.3|
|    Noida|    28.5|     77.4|
|  Manipal|    13.4|     74.8|
|    Udupi|    13.3|     74.7|
+---------+--------+---------+



Create and fit the pipeline

In [3]:
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.clustering import KMeans
from pyspark.ml import Pipeline

assembler = VectorAssembler(inputCols=["latitude", "longitude"], outputCol="featureVector")
scaler = StandardScaler(inputCol="featureVector", outputCol="scaledFeatureVector")
classifier = KMeans(featuresCol="scaledFeatureVector", predictionCol="cluster", k=2)

pipeline = Pipeline(stages=[assembler, scaler, classifier])
pipeline_model = pipeline.fit(df)

In [4]:
kmeans_model = pipeline_model.stages[-1]
kmeans_model.summary.trainingCost

0.0656029904280815

Set threshold for anomalies

In [40]:
import numpy as np
from pyspark.sql.functions import udf, col

centroids = kmeans_model.clusterCenters()
clustered = pipeline_model.transform(df)

@udf
def dist(cluster, vector):
    return float(vector.squared_distance(centroids[cluster]))

clustered \
    .withColumn("distance", dist("cluster", "scaledFeatureVector")) \
    .select("cluster", "scaledFeatureVector") \
    .orderBy(col("distance").desc()) \
    .show(5)

+-------+--------------------+
|cluster| scaledFeatureVector|
+-------+--------------------+
|      0|[3.63659274466529...|
|      0|[3.63659274466529...|
|      0|[3.66211269375066...|
|      0|[3.62383277012261...|
|      1|[1.70983658871982...|
+-------+--------------------+
only showing top 5 rows

